### Weather, Traffic, Time, and categorical values  feature engineering is done in this file

In [80]:
#import needed libraries & float format definition
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from scipy import stats
from scipy.stats import norm
from matplotlib import pyplot
from matplotlib.pyplot import figure
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.metrics import mean_squared_error
import time
import datetime

In [101]:
#Loading Traffic Data 
Traffic_Data = pd.read_csv('Feature_E_MergePython/Traffic Data/Traffic_Data.csv')
Traffic_Data = Traffic_Data.rename(columns = {'Unnamed: 0':'index'})
Traffic_Data

,index,fl_date,origin_city_name,dest_city_name,Trip_count_PerDay,Trip_count_Permonth,Trip_count_PerYear
0,0,2018-01-01,"Tampa, FL","Albany, NY",1.001351,30.875000,61.750000
1,1,2018-01-01,"New Orleans, LA","Houston, TX",17.332432,534.416667,1068.833333
2,2,2018-01-01,"New Orleans, LA","Orlando, FL",5.255405,162.041667,324.083333
3,3,2018-01-01,"New Orleans, LA","Oakland, CA",0.959459,29.583333,59.166667
4,4,2018-01-01,"New Orleans, LA","St. Louis, MO",1.981081,61.083333,122.166667
...,...,...,...,...,...,...,...
14595,14595,2019-12-31,"New York, NY","Kansas City, MO",5.506757,169.791667,339.583333
14596,14596,2019-12-31,"New York, NY","Chicago, IL",54.166216,1670.125000,3340.250000
14597,14597,2019-12-31,"New York, NY","New Orleans, LA",8.470270,261.166667,522.333333
14598,14598,2019-12-31,"New York, NY","Tampa, FL",12.725676,392.375000,784.750000


In [82]:
#Loading Flights Data 
Flights = pd.read_csv('Flights_Data/Flights_OriginalTable.csv')
Flights = Flights.rename(columns = {'Unnamed: 0':'index'})
Flights 

,index,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin_city_name,dest_city_name,crs_dep_time,dep_delay,taxi_out,wheels_off,...,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,diverted,crs_elapsed_time,actual_elapsed_time,air_time,distance
0,0,2018-01-01,WN,5705,"Tampa, FL","Albany, NY",1530,28.0,12.0,1610.0,...,7.0,1810,1831.0,21.0,0,0,160,153.0,134.0,1130
1,1,2018-01-01,WN,988,"New Orleans, LA","Houston, TX",625,-2.0,9.0,632.0,...,3.0,740,732.0,-8.0,0,0,75,69.0,57.0,302
2,2,2018-01-01,WN,1236,"New Orleans, LA","Orlando, FL",515,-1.0,8.0,522.0,...,8.0,755,740.0,-15.0,0,0,100,86.0,70.0,551
3,3,2018-01-01,WN,233,"New Orleans, LA","Oakland, CA",1530,16.0,8.0,1554.0,...,4.0,1820,1829.0,9.0,0,0,290,283.0,271.0,1903
4,4,2018-01-01,WN,446,"New Orleans, LA","St. Louis, MO",1805,3.0,6.0,1814.0,...,36.0,1950,2013.0,23.0,0,0,105,125.0,83.0,604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14595,14595,2019-12-31,WN,5451,"New York, NY","Kansas City, MO",1835,94.0,7.0,2016.0,...,3.0,2100,2154.0,54.0,0,0,205,165.0,155.0,1107
14596,14596,2019-12-31,WN,2038,"New York, NY","Chicago, IL",1155,-4.0,13.0,1204.0,...,44.0,1335,1336.0,1.0,0,0,160,165.0,108.0,725
14597,14597,2019-12-31,WN,3255,"New York, NY","New Orleans, LA",1235,0.0,16.0,1251.0,...,3.0,1510,1453.0,-17.0,0,0,215,198.0,179.0,1183
14598,14598,2019-12-31,WN,4880,"New York, NY","Tampa, FL",845,10.0,12.0,907.0,...,4.0,1150,1149.0,-1.0,0,0,185,174.0,158.0,1010


In [83]:
#Merging Traffic Data to Flights
Flights_Traffic = pd.merge(Flights, Traffic_Data, how ='left', on = ['index','fl_date','origin_city_name','dest_city_name'])
Flights_Traffic 

,index,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin_city_name,dest_city_name,crs_dep_time,dep_delay,taxi_out,wheels_off,...,arr_delay,cancelled,diverted,crs_elapsed_time,actual_elapsed_time,air_time,distance,Trip_count_PerDay,Trip_count_Permonth,Trip_count_PerYear
0,0,2018-01-01,WN,5705,"Tampa, FL","Albany, NY",1530,28.0,12.0,1610.0,...,21.0,0,0,160,153.0,134.0,1130,1.001351,30.875000,61.750000
1,1,2018-01-01,WN,988,"New Orleans, LA","Houston, TX",625,-2.0,9.0,632.0,...,-8.0,0,0,75,69.0,57.0,302,17.332432,534.416667,1068.833333
2,2,2018-01-01,WN,1236,"New Orleans, LA","Orlando, FL",515,-1.0,8.0,522.0,...,-15.0,0,0,100,86.0,70.0,551,5.255405,162.041667,324.083333
3,3,2018-01-01,WN,233,"New Orleans, LA","Oakland, CA",1530,16.0,8.0,1554.0,...,9.0,0,0,290,283.0,271.0,1903,0.959459,29.583333,59.166667
4,4,2018-01-01,WN,446,"New Orleans, LA","St. Louis, MO",1805,3.0,6.0,1814.0,...,23.0,0,0,105,125.0,83.0,604,1.981081,61.083333,122.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14595,14595,2019-12-31,WN,5451,"New York, NY","Kansas City, MO",1835,94.0,7.0,2016.0,...,54.0,0,0,205,165.0,155.0,1107,5.506757,169.791667,339.583333
14596,14596,2019-12-31,WN,2038,"New York, NY","Chicago, IL",1155,-4.0,13.0,1204.0,...,1.0,0,0,160,165.0,108.0,725,54.166216,1670.125000,3340.250000
14597,14597,2019-12-31,WN,3255,"New York, NY","New Orleans, LA",1235,0.0,16.0,1251.0,...,-17.0,0,0,215,198.0,179.0,1183,8.470270,261.166667,522.333333
14598,14598,2019-12-31,WN,4880,"New York, NY","Tampa, FL",845,10.0,12.0,907.0,...,-1.0,0,0,185,174.0,158.0,1010,12.725676,392.375000,784.750000


In [84]:
#Loading Weather Data 
path = 'Feature_E_MergePython/Weather_Data/final_origin_dest_weather.csv'
Weather_Data = pd.read_csv(path)
Weather_Data = Weather_Data.rename(columns = {'Unnamed: 0':'index'})
Weather_Data

,index,fl_date,origin_city_name,dest_city_name,origin_city_condn,dest_city_condn
0,0,2018-01-01,"Tampa, FL","Albany, NY",low_rainny,sunny
1,1,2018-01-01,"New Orleans, LA","Houston, TX",cloundy,sunny
2,2,2018-01-01,"New Orleans, LA","Orlando, FL",cloundy,low_rainny
3,3,2018-01-01,"New Orleans, LA","Oakland, CA",cloundy,sunny
4,4,2018-01-01,"New Orleans, LA","St. Louis, MO",cloundy,sunny
...,...,...,...,...,...,...
14595,14595,2019-12-31,"New York, NY","Kansas City, MO",cloundy,sunny
14596,14596,2019-12-31,"New York, NY","Chicago, IL",cloundy,low_snow
14597,14597,2019-12-31,"New York, NY","New Orleans, LA",cloundy,sunny
14598,14598,2019-12-31,"New York, NY","Tampa, FL",cloundy,sunny


In [99]:
#Merging API weather Data to the Flighs_Traffic tables 
Merged_Table_1 = pd.merge(Flights_Traffic, Weather_Data, how ='left', on = ['index','fl_date','origin_city_name','dest_city_name'])
Merged_Table_1 

,index,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin_city_name,dest_city_name,crs_dep_time,dep_delay,taxi_out,wheels_off,...,diverted,crs_elapsed_time,actual_elapsed_time,air_time,distance,Trip_count_PerDay,Trip_count_Permonth,Trip_count_PerYear,origin_city_condn,dest_city_condn
0,0,2018-01-01,WN,5705,"Tampa, FL","Albany, NY",1530,28.0,12.0,1610.0,...,0,160,153.0,134.0,1130,1.001351,30.875000,61.750000,low_rainny,sunny
1,1,2018-01-01,WN,988,"New Orleans, LA","Houston, TX",625,-2.0,9.0,632.0,...,0,75,69.0,57.0,302,17.332432,534.416667,1068.833333,cloundy,sunny
2,2,2018-01-01,WN,1236,"New Orleans, LA","Orlando, FL",515,-1.0,8.0,522.0,...,0,100,86.0,70.0,551,5.255405,162.041667,324.083333,cloundy,low_rainny
3,3,2018-01-01,WN,233,"New Orleans, LA","Oakland, CA",1530,16.0,8.0,1554.0,...,0,290,283.0,271.0,1903,0.959459,29.583333,59.166667,cloundy,sunny
4,4,2018-01-01,WN,446,"New Orleans, LA","St. Louis, MO",1805,3.0,6.0,1814.0,...,0,105,125.0,83.0,604,1.981081,61.083333,122.166667,cloundy,sunny
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14595,14595,2019-12-31,WN,5451,"New York, NY","Kansas City, MO",1835,94.0,7.0,2016.0,...,0,205,165.0,155.0,1107,5.506757,169.791667,339.583333,cloundy,sunny
14596,14596,2019-12-31,WN,2038,"New York, NY","Chicago, IL",1155,-4.0,13.0,1204.0,...,0,160,165.0,108.0,725,54.166216,1670.125000,3340.250000,cloundy,low_snow
14597,14597,2019-12-31,WN,3255,"New York, NY","New Orleans, LA",1235,0.0,16.0,1251.0,...,0,215,198.0,179.0,1183,8.470270,261.166667,522.333333,cloundy,sunny
14598,14598,2019-12-31,WN,4880,"New York, NY","Tampa, FL",845,10.0,12.0,907.0,...,0,185,174.0,158.0,1010,12.725676,392.375000,784.750000,cloundy,sunny


In [100]:
Merged_Table_1['Minutes_to_wheels_off'] = Merged_Table_1['wheels_off'] - Merged_Table_1['crs_dep_time']
Merged_Table_1

,index,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin_city_name,dest_city_name,crs_dep_time,dep_delay,taxi_out,wheels_off,...,crs_elapsed_time,actual_elapsed_time,air_time,distance,Trip_count_PerDay,Trip_count_Permonth,Trip_count_PerYear,origin_city_condn,dest_city_condn,Minutes_to_wheels_off
0,0,2018-01-01,WN,5705,"Tampa, FL","Albany, NY",1530,28.0,12.0,1610.0,...,160,153.0,134.0,1130,1.001351,30.875000,61.750000,low_rainny,sunny,80.0
1,1,2018-01-01,WN,988,"New Orleans, LA","Houston, TX",625,-2.0,9.0,632.0,...,75,69.0,57.0,302,17.332432,534.416667,1068.833333,cloundy,sunny,7.0
2,2,2018-01-01,WN,1236,"New Orleans, LA","Orlando, FL",515,-1.0,8.0,522.0,...,100,86.0,70.0,551,5.255405,162.041667,324.083333,cloundy,low_rainny,7.0
3,3,2018-01-01,WN,233,"New Orleans, LA","Oakland, CA",1530,16.0,8.0,1554.0,...,290,283.0,271.0,1903,0.959459,29.583333,59.166667,cloundy,sunny,24.0
4,4,2018-01-01,WN,446,"New Orleans, LA","St. Louis, MO",1805,3.0,6.0,1814.0,...,105,125.0,83.0,604,1.981081,61.083333,122.166667,cloundy,sunny,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14595,14595,2019-12-31,WN,5451,"New York, NY","Kansas City, MO",1835,94.0,7.0,2016.0,...,205,165.0,155.0,1107,5.506757,169.791667,339.583333,cloundy,sunny,181.0
14596,14596,2019-12-31,WN,2038,"New York, NY","Chicago, IL",1155,-4.0,13.0,1204.0,...,160,165.0,108.0,725,54.166216,1670.125000,3340.250000,cloundy,low_snow,49.0
14597,14597,2019-12-31,WN,3255,"New York, NY","New Orleans, LA",1235,0.0,16.0,1251.0,...,215,198.0,179.0,1183,8.470270,261.166667,522.333333,cloundy,sunny,16.0
14598,14598,2019-12-31,WN,4880,"New York, NY","Tampa, FL",845,10.0,12.0,907.0,...,185,174.0,158.0,1010,12.725676,392.375000,784.750000,cloundy,sunny,62.0


In [86]:
Merged_Table_1.to_csv('Merged_Table_1.csv')

In [87]:
# Merged_Table_1.columns()

In [88]:
Merged_Table_1['crs_dep_time'].dtype

dtype('int64')

In [89]:
Merged_Table_1.crs_dep_time .dtype
# Might need this again for other variables so heres a function 
def TimeFormatted(hours):
    """Changing times from int format into HH:MM format """
    if hours == 2400:
        hours = 0
    else:
        hours = "{0:04d}".format(int(hours))
        Hourmin = datetime.time(int(hours[0:2]), int(hours[2:4]))
        return Hourmin

In [91]:
Merged_Table_1['crs_dep_time'] = Merged_Table_1['crs_dep_time'].apply(TimeFormatted)
Merged_Table_1['crs_arr_time'] = Merged_Table_1['crs_arr_time'].apply(TimeFormatted)

In [92]:
Merged_Table_1['crs_dep_time'] = pd.to_datetime(Merged_Table_1['crs_dep_time'],format = '%H:%M:%S').dt.hour
Merged_Table_1['crs_arr_time'] = pd.to_datetime(Merged_Table_1['crs_arr_time'],format = '%H:%M:%S').dt.hour

In [94]:
Merged_Table_1.to_csv('Merged_Table_1.csv')

In [95]:
Merged_Table_1.head()

,index,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin_city_name,dest_city_name,crs_dep_time,dep_delay,taxi_out,wheels_off,...,diverted,crs_elapsed_time,actual_elapsed_time,air_time,distance,Trip_count_PerDay,Trip_count_Permonth,Trip_count_PerYear,origin_city_condn,dest_city_condn
0,0,2018-01-01,WN,5705,"Tampa, FL","Albany, NY",15,28.0,12.0,1610.0,...,0,160,153.0,134.0,1130,1.001351,30.875000,61.750000,low_rainny,sunny
1,1,2018-01-01,WN,988,"New Orleans, LA","Houston, TX",6,-2.0,9.0,632.0,...,0,75,69.0,57.0,302,17.332432,534.416667,1068.833333,cloundy,sunny
2,2,2018-01-01,WN,1236,"New Orleans, LA","Orlando, FL",5,-1.0,8.0,522.0,...,0,100,86.0,70.0,551,5.255405,162.041667,324.083333,cloundy,low_rainny
3,3,2018-01-01,WN,233,"New Orleans, LA","Oakland, CA",15,16.0,8.0,1554.0,...,0,290,283.0,271.0,1903,0.959459,29.583333,59.166667,cloundy,sunny
4,4,2018-01-01,WN,446,"New Orleans, LA","St. Louis, MO",18,3.0,6.0,1814.0,...,0,105,125.0,83.0,604,1.981081,61.083333,122.166667,cloundy,sunny


In [96]:
Merged_Table_1.columns

Index(['index', 'fl_date', 'mkt_unique_carrier', 'mkt_carrier_fl_num',
       'origin_city_name', 'dest_city_name', 'crs_dep_time', 'dep_delay',
       'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in', 'crs_arr_time',
       'arr_time', 'arr_delay', 'cancelled', 'diverted', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'distance', 'Trip_count_PerDay',
       'Trip_count_Permonth', 'Trip_count_PerYear', 'origin_city_condn',
       'dest_city_condn'],
      dtype='object')